# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

I. Momcheva  ->  I. Momcheva  |  ['I. Momcheva']
F. Walter  ->  F. Walter  |  ['F. Walter']
F. Davies  ->  F. Davies  |  ['F. Davies']
P. Gaikwad  ->  P. Gaikwad  |  ['P. Gaikwad']
S. Rojas-Ruiz  ->  S. Rojas-Ruiz  |  ['S. Rojas-Ruiz']
J. He  ->  J. He  |  ['J. He']


S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
Arxiv has 56 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2305.04953


extracting tarball to tmp_2305.04953...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

extracting tarball to tmp_2305.05053...

 done.


Found 160 bibliographic references in tmp_2305.05053/XQR30_general_arxiv.bbl.
Retrieving document from  https://arxiv.org/e-print/2305.05372


extracting tarball to tmp_2305.05372...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.04953-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.04953) | **CLEAR: The Morphological Evolution of Galaxies in the Green Valley**  |
|| V. Estrada-Carpenter, et al. -- incl., <mark>I. Momcheva</mark> |
|*Appeared on*| *2023-05-10*|
|*Comments*| *17 pages, 10 figures, submitted to ApJ*|
|**Abstract**| Quiescent galaxies having more compact morphologies than star-forming galaxies has been a consistent result in the field of galaxy evolution. What is not clear is at what point this divergence happens, i.e. when do quiescent galaxies become compact, and how big of a role does the progenitor effect play in this result? Here we aim to model the morphological and star-formation histories of high redshift (0.8 $<$ z $<$ 1.65) massive galaxies (log(M/M$\odot$) $>$ 10.2) with stellar population fits using HST/WFC3 G102 and G141 grism spectra plus photometry from the CLEAR (CANDELS Lyman-alpha Emission at Reionization) survey, constraining the star-formation histories for a sample of $\sim$ 400 massive galaxies using flexible star-formation histories. We develop a novel approach to classifying galaxies by their formation activity in a way that highlights the green valley population, by modeling the specific star-formation rate distributions as a function of redshift and deriving the probability that a galaxy is quiescent (PQ). Using PQ and our flexible star-formation histories we outline the evolutionary paths of our galaxies in relation to stellar mass, Sersic index, $R_{eff}$, and stellar mass surface density. We find that galaxies show no appreciable stellar mass growth after entering the green valley (a net decrease of 4$\%$) while their stellar mass surface densities increase by $\sim$ 0.25 dex. Therefore galaxies are becoming compact during the green valley and this is due to increases in Sersic index and decreases in $R_{eff}$. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.05053-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.05053) | **XQR-30: the ultimate XSHOOTER quasar sample at the reionization epoch**  |
|| V. D'Odorico, et al. -- incl., <mark>F. Walter</mark>, <mark>F. Davies</mark>, <mark>P. Gaikwad</mark>, <mark>S. Rojas-Ruiz</mark> |
|*Appeared on*| *2023-05-10*|
|*Comments*| *21 pages, 10 figures. Revised version resubmitted to MNRAS after minor referee report*|
|**Abstract**| The final phase of the reionization process can be probed by rest-frame UV absorption spectra of quasars at z>6, shedding light on the properties of the diffuse intergalactic medium within the first Gyr of the Universe. The ESO Large Programme "XQR-30: the ultimate XSHOOTER legacy survey of quasars at z~5.8-6.6" dedicated ~250 hours of observations at the VLT to create a homogeneous and high-quality sample of spectra of 30 luminous quasars at z~6, covering the rest wavelength range from the Lyman limit to beyond the MgII emission. Twelve quasar spectra of similar quality from the XSHOOTER archive were added to form the enlarged XQR-30 sample, corresponding to a total of ~350 hours of on-source exposure time. The median effective resolving power of the 42 spectra is R~11400 and 9800 in the VIS and NIR arm, respectively. The signal-to-noise ratio per 10 km/s pixel ranges from ~11 to 114 at $\lambda \simeq 1285$ \AA rest frame, with a median value of ~29. We describe the observations, data reduction and analysis of the spectra, together with some first results based on the E-XQR-30 sample. New photometry in the H and K bands are provided for the XQR-30 quasars, together with composite spectra whose characteristics reflect the large absolute magnitudes of the sample. The composite and the reduced spectra are released to the community through a public repository, and will enable a range of studies addressing outstanding questions regarding the first Gyr of the Universe. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.05372-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.05372) | **Measurement of ultra-high-energy diffuse gamma-ray emission of the  Galactic plane from 10 TeV to 1 PeV with LHAASO-KM2A**  |
|| Z. Cao, et al. -- incl., <mark>J. He</mark> |
|*Appeared on*| *2023-05-10*|
|*Comments*| *11 pages, 7 figures, 5 tables*|
|**Abstract**| The diffuse Galactic $\gamma$-ray emission, mainly produced via interactions between cosmic rays and the diffuse interstellar medium, is a very important probe of the distribution, propagation, and interaction of cosmic rays in the Milky Way. In this work we report the measurements of diffuse $\gamma$-rays from the Galactic plane between 10 TeV and 1 PeV energies, with the square kilometer array of the Large High Altitude Air Shower Observatory (LHAASO). Diffuse emissions from the inner ($15^{\circ}<l<125^{\circ}$, $|b|<5^{\circ}$) and outer ($125^{\circ}<l<235^{\circ}$, $|b|<5^{\circ}$) Galactic plane are detected with $29.1\sigma$ and $12.7\sigma$ significance, respectively. The outer Galactic plane diffuse emission is detected for the first time in the very- to ultra-high-energy domain ($E>10$~TeV). The energy spectrum in the inner Galaxy regions can be described by a power-law function with an index of $-2.99\pm0.04$, which is different from the curved spectrum as expected from hadronic interactions between locally measured cosmic rays and the line-of-sight integrated gas content. Furthermore, the measured flux is higher by a factor of $\sim3$ than the prediction. A similar spectrum with an index of $-2.99\pm0.07$ is found in the outer Galaxy region, and the absolute flux for $10\lesssim E\lesssim60$ TeV is again higher than the prediction for hadronic cosmic ray interactions. The latitude distributions of the diffuse emission are consistent with the gas distribution, while the longitude distributions show slight deviation from the gas distribution. The LHAASO measurements imply that either additional emission sources exist or cosmic ray intensities have spatial variations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2305.04953.md
exported in  _build/html/2305.05053.md
    + _build/html/tmp_2305.05053/./FiguraJ_z.png
    + _build/html/tmp_2305.05053/./composite-spectrum-noBAL-nopDLA.png
    + _build/html/tmp_2305.05053/./J0100_spFWHM_vs_DIMM_VIS.png
    + _build/html/tmp_2305.05053/./J0100_tellFWHM_vs_DIMM_VIS.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

.
 (*fig_Psf*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.04953"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\CIV}{\mbox{C {\sc iv}}}$
$\newcommand{\CV}{\mbox{C {\sc v}}}$
$\newcommand{\CIII}{\mbox{C {\sc iii}}}$
$\newcommand{\CII}{\mbox{C {\sc ii}}}$
$\newcommand{\NV}{\mbox{N {\sc v}}}$
$\newcommand{\NIII}{\mbox{N {\sc iii}}}$
$\newcommand{\NII}{\mbox{N {\sc ii}}}$
$\newcommand{\OVI}{\mbox{O {\sc vi}}}$
$\newcommand{\OIV}{\mbox{O {\sc iv}}}$
$\newcommand{\OIII}{\mbox{O {\sc iii}}}$
$\newcommand{\OII}{\mbox{O {\sc ii}}}$
$\newcommand{\OI}{\mbox{O {\sc i}}}$
$\newcommand{\MgI}{\mbox{Mg {\sc i}}}$
$\newcommand{\MgII}{\mbox{Mg {\sc ii}}}$
$\newcommand{\AlII}{\mbox{Al {\sc ii}}}$
$\newcommand{\AlIII}{\mbox{Al {\sc iii}}}$
$\newcommand{\SiIV}{\mbox{Si {\sc iv}}}$
$\newcommand{\SiIII}{\mbox{Si {\sc iii}}}$
$\newcommand{\SiII}{\mbox{Si {\sc ii}}}$
$\newcommand{\SiV}{\mbox{Si {\sc v}}}$
$\newcommand{\SIII}{\mbox{S {\sc iii}}}$
$\newcommand{\SIV}{\mbox{S {\sc iv}}}$
$\newcommand{\FeIII}{\mbox{Fe {\sc iii}}}$
$\newcommand{\FeII}{\mbox{Fe {\sc ii}}}$
$\newcommand{\ZnII}{\mbox{Zn {\sc ii}}}$
$\newcommand{\HI}{\mbox{H {\sc i}}}$
$\newcommand{\HII}{\mbox{H {\sc ii}}}$
$\newcommand{\HeII}{\mbox{He {\sc ii}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\gsim}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\lsim}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\zabs}{z_{\rm abs}}$
$\newcommand{\zem}{z_{\rm em}}$
$\newcommand{\zlb}{z_{\rm Ly\beta}}$
$\newcommand{\Lya}{Ly\alpha}$
$\newcommand{\Lyb}{Ly\beta}$
$\newcommand{\lam}{\lambda}$
$\newcommand{\lama}{\lambda_{\alpha}}$
$\newcommand{\lamb}{\lambda_{\beta}}$
$\newcommand{\lambdaem}{\lambda_{\rm em}~}$
$\newcommand{\Jv}{{\rm J_{\nu}~}}$
$\newcommand{\nh}{{n_{\rm H}~}}$
$\newcommand{\nhs}{{\rm n_{\rm H,syn}~}}$
$\newcommand{\LJ}{{L_{\rm J}}}$
$\newcommand{\NHI}{{N_{\rm HI}}}$</div>



<div id="title">

# XQR-30: the ultimate XSHOOTER quasar sample at the reionization epoch

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.05053-b31b1b.svg)](https://arxiv.org/abs/2305.05053)<mark>Appeared on: 2023-05-10</mark> -  _21 pages, 10 figures. Revised version resubmitted to MNRAS after minor referee report_

</div>
<div id="authors">

V. D'Odorico, et al. -- incl., <mark>F. Walter</mark>, <mark>F. Davies</mark>, <mark>P. Gaikwad</mark>, <mark>S. Rojas-Ruiz</mark>

</div>
<div id="abstract">

**Abstract:** The final phase of the reionization process can be probed by rest--frame UV absorption spectra of quasars at $z>6$ , shedding light on the properties of the diffuse intergalactic medium within the first Gyr of the Universe. The ESO Large Programme “XQR-30: the ultimate XSHOOTER legacy survey of quasars at $z \simeq 5.8-6.6$ ” dedicated $\sim250$ hours of observations at the VLT to create a homogeneous and high-quality sample of spectra of 30 luminous quasars at $z\sim6$ , covering the rest wavelength range from the Lyman limit to beyond the $\MgII$ emission. Twelve quasar spectra of similar quality from the XSHOOTER archive were added to form the enlarged XQR-30 sample, corresponding to a total of $\sim350$ hours of on-source exposure time. The median effective resolving power of the  42 spectra is $R \simeq 11400$ and $9800$ in the VIS and NIR arm, respectively. The signal-to-noise ratio per 10 $\kms$ pixel ranges from $\sim11$ to 114 at $\lambda\simeq 1285$ Å rest frame, with a median value of $\sim29$ . We describe the observations, data reduction and analysis of the spectra, together with some first results based on the E-XQR-30 sample. New photometry in the $H$ and $K$ bands are provided for the XQR-30 quasars, together with composite spectra whose characteristics reflect the large absolute magnitudes of the sample.  The composite and the reduced spectra are released to the community through a public repository, and will enable a range of studies addressing outstanding questions regarding the first Gyr of the Universe.

</div>

<div id="div_fig1">

<img src="tmp_2305.05053/./FiguraJ_z.png" alt="Fig2" width="100%"/>

**Figure 2. -** Distribution of the XQR-30 targets (blue squares) in the Redshift-$J_{\rm AB}$ magnitude plane. The orange dots represent the 9 quasars with SNR $ \ge 25$ per pixel, already available from the XSHOOTER archive at the time of the proposal preparation. The green diamonds are the 4 quasars with similar properties whose spectra in the XSHOOTER archive became available after 2018. VDES J0224-4711 is represented as a green diamond with a blue contour (see text). (*fig:redshift_range*)

</div>
<div id="div_fig2">

<img src="tmp_2305.05053/./composite-spectrum-noBAL-nopDLA.png" alt="Fig14" width="100%"/>

**Figure 14. -** _Upper panel:_ Composite spectrum of the total E-XQR-30 sample rebinned to 250 $\kms$(black curve), compared with the composite spectrum obtained excluding the targets showing BAL systems (orange curve), as detected in [ and Bischetti (2022)](https://ui.adsabs.harvard.edu/abs/2022Natur.605..244B). The list of targets contributing to the no-BAL composite is reported in Table \ref{tab:noBAL}. _Lower panel:_ Composite spectrum of the total E-XQR-30 sample rebinned to 250 $\kms$(black curve), compared with the composite spectrum obtained excluding the targets showing proximate DLAs (blue curve), as identified in [ and Davies (2023)]() and Sodini et al. in prep. The list of targets contributing to the no-pDLA composite is reported in Table \ref{tab:nopDLA}. (*fig:comp_noBAL_nopDLA*)

</div>
<div id="div_fig3">

<img src="tmp_2305.05053/./J0100_spFWHM_vs_DIMM_VIS.png" alt="Fig4.1" width="50%"/><img src="tmp_2305.05053/./J0100_tellFWHM_vs_DIMM_VIS.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** FWHM of the spectral order spatial profiles (_ upper panel_) and average FWHM of the telluric model (_ lower panel_) as a function of the FWHM at $\sim950$ nm of the seeing disk for the VIS frames of the literature quasar SDSSJ0100+2802,  which was observed with a slit=0.9". The dots are coloured according to the SNR of the corresponding frame (see the scale in the sidebar). The dashed lines indicate the slit value.   (*fig:resol_VIS_J0100*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.05053"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

204  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
